# Advent of Code 2021 - PowerShell

## Day 4 - Giant Squid

[Puzzle Link](https://adventofcode.com/2021/day/4)

### Input

In [ ]:
$UseSampleData = $true
$DataFile = $UseSampleData ? '..\PuzzleInput\day4_sample.txt' : '..\PuzzleInput\day4.txt'

$BingoData = Get-Content -Path $DataFile

### Puzzle 1

What will be your final score for the winning board?

In [ ]:
class BingoCard {
    [int[,]]$Numbers = [int[,]]::CreateInstance([int],5,5)
    [bool[,]]$Marked = [bool[,]]::CreateInstance([bool],5,5)

    BingoCard([string[]]$CardNumbers) {
        for ($row = 0;$row -lt $CardNumbers.Count;$row++) {
            [string[]]$RowNumbers = $CardNumbers[$row].split(' ',[System.StringSplitOptions]::RemoveEmptyEntries)
            for ($col = 0; $col -lt $RowNumbers.Count;$col++) {
                $this.Numbers[$row,$col] = $RowNumbers[$col]
            }
        }
    }

    [void] Mark([int]$CalledNumber) {
        for ($row = 0; $row -lt 5; $row++) {
            for ($col = 0; $col -lt 5; $col++) {
                if ($this.Numbers[$row, $col] -eq $CalledNumber) {
                    $this.Marked[$row, $col] = $true
                }
            }
        }
    }

    [bool] HasBingo() {
        for ($row = 0; $row -lt 5; $row++) {
            if ($this.Marked[$row, 0] -and $this.Marked[$row, 1] -and
                $this.Marked[$row, 2] -and $this.Marked[$row, 3] -and
                $this.Marked[$row, 4]) {
                return $true
            }
        }
        for ($col = 0; $col -lt 5; $col++) {
            if ($this.Marked[0, $col] -and $this.Marked[1, $col] -and
                $this.Marked[2, $col] -and $this.Marked[3, $col] -and
                $this.Marked[4, $col]) {
                return $true
            }
        }
        return $false
    }
    
    [int] SumUnmarked() {
        $sum = 0
        for ($col = 0; $col -lt 5; $col++) {
            for ($row = 0; $row -lt 5; $row++) {
                if (-not $this.Marked[$row, $col]) {
                    $sum += $this.Numbers[$row, $col]
                }
            }
        }
        return $sum
    }

    [string] Results([int]$Card,[int]$Number,[string]$FirstLast) {
        $Sum = $this.SumUnmarked()
        $Results = '{0} Winner Board: {1}, Winning Number: {2}, Score: {3}' -f $FirstLast,$Card,$Number,($Sum * $Number)
        return $Results
    }
}

$BingoCards = for ($row = 2; $row -lt $BingoData.Count;$row+=6) {
    [BingoCard]::new($BingoData[$row..($row+4)])
}

[int[]]$BingoNumbers = $BingoData[0].Split(',')

:CallNumbers foreach ($Number in $BingoNumbers) {
    for ($Card = 0; $Card -lt $BingoCards.Count; $Card++) {
        $BingoCards[$Card].Mark($Number)
        if ($BingoCards[$Card].HasBingo()) {
            $BingoCards[$Card].Results($Card,$Number,'First')
            break CallNumbers
        }        
    }
}

# Sample: First Winner Board: 2, Winning Number: 24, Score: 4512
# Answer: First Winner Board: 53, Winning Number: 35, Score: 25410

First Winner Board: 2, Winning Number: 24, Score: 4512


### Puzzle 2

Figure out which board will win last. Once it wins, what would its final score be?

In [ ]:
# reset bingo cards
$BingoCards = for ($row = 2; $row -lt $BingoData.Count;$row+=6) {
    [BingoCard]::new($BingoData[$row..($row+4)])
}

$WinningCards = @{}
:CallNumbers foreach ($Number in $BingoNumbers) {
    for ($Card = 0; $Card -lt $BingoCards.Count; $Card++) {
        if ($WinningCards.ContainsKey($Card)) {
            continue
        }
        $BingoCards[$Card].Mark($Number)
        if ($BingoCards[$Card].HasBingo()) {
            $WinningCards[$Card] = $true
            if ($WinningCards.Count -eq $BingoCards.Count) {                                
                $BingoCards[$Card].Results($Card,$Number,'Last')
                break CallNumbers
            }
        }        
    }
}

# Sample: Last Winner Board: 1, Winning Number: 13, Score: 1924
# Answer: Last Winner Board: 40, Winning Number: 14, Score: 2730

Last Winner Board: 1, Winning Number: 13, Score: 1924
